# LEAVE and ITERATE Statements

**Applies to:** Databricks SQL, Databricks Runtime 16.3+

- **LEAVE label** – Terminates the loop and exits (like `break`). The label must identify a looping statement (LOOP, WHILE, REPEAT, FOR) that contains the LEAVE.
- **ITERATE label** – Skips the rest of the current iteration and continues with the next (like `continue`). Same label rules.

Both may only be used inside a compound statement. The **label** must match the label on the loop.

In [0]:
USE CATALOG healthcare_dev;
USE SCHEMA gold;

## Example: LEAVE – exit FOR loop when running total exceeds threshold (fact_patient_claims)

In [0]:
BEGIN
  DECLARE running DECIMAL(18,2) DEFAULT 0;
  DECLARE threshold DECIMAL(18,2) DEFAULT 10000;
  sumLoop: FOR row AS
    SELECT clm_pmt_amt AS amt FROM fact_cms_outpatient_claims WHERE clm_pmt_amt IS NOT NULL LIMIT 500
  DO
    SET running = running + COALESCE(row.amt, 0);
    IF running >= threshold THEN
      LEAVE sumLoop;
    END IF;
  END FOR sumLoop;
  SELECT running AS total_at_exit, threshold;
END

## Example: ITERATE – process only rows where amount > 0 (skip zeros)

In [0]:
BEGIN
  DECLARE nonzero_count INT DEFAULT 0;
  DECLARE total_amt DECIMAL(18,2) DEFAULT 0;
  procLoop: FOR r AS
    SELECT clm_pmt_amt AS amt FROM fact_cms_outpatient_claims LIMIT 100
  DO
    IF r.amt IS NULL OR r.amt <= 0 THEN
      ITERATE procLoop;
    END IF;
    SET nonzero_count = nonzero_count + 1;
    SET total_amt = total_amt + r.amt;
  END FOR procLoop;
  SELECT nonzero_count, total_amt;
END